# 1. Import packages

In [60]:
import copy
import ipaddress
import numpy as np
import os
import pandas as pd
import re
import time

from datetime import timedelta
from tld import get_fld

# 2. Read a sample CSV

## 2.1. Change to inputs directory

In [2]:
print(os.getcwd())
input_path_s = "/input/"
input_path_s = os.getcwd() + input_path_s
output_path_s = "/output/"
output_path_s = os.getcwd() + output_path_s
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain
/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/input


## 2.2. Read CSV: 2022-08-09.csv

In [3]:
bright_data_df = pd.read_csv("domains_bright_data.csv")

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr


In [5]:
bright_data_df.shape

(1075319, 4)

## 2.3. Read tld list

In [6]:
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/input


In [41]:
two_letter_tld_set = set()

# I am pretty sure there is a way to parse and read at the same time, instead of reading all at once.
tld_l = list(pd.read_csv("tld-list-basic.txt", header=None)[0])
for tld in tld_l:
    if len(tld) == 2:
        two_letter_tld_set.add("." + tld)

two_letter_tld_set -= set([".ai", ".io"])
print(len(two_letter_tld_set))
print(two_letter_tld_set)

317
{'.jp', '.bh', '.nu', '.pm', '.eh', '.md', '.ad', '.微博', '.uy', '.台湾', '.ευ', '.广东', '.游戏', '.mk', '.bq', '.cy', '.bw', '.网站', '.hu', '.გე', '.vi', '.pn', '.hn', '.рф', '.ie', '.bt', '.cg', '.bz', '.fr', '.中信', '.mz', '.ro', '.it', '.集团', '.ng', '.bv', '.时尚', '.购物', '.bd', '.as', '.eu', '.ae', '.dz', '.测试', '.中国', '.网店', '.rw', '.de', '.gf', '.gs', '.pg', '.cx', '.iq', '.ms', '.sh', '.佛山', '.cl', '.sr', '.sm', '.er', '.td', '.mt', '.コム', '.通販', '.mg', '.sn', '.大拿', '.im', '.sy', '.me', '.bn', '.ir', '.gm', '.sv', '.信息', '.ba', '.gp', '.in', '.cr', '.zm', '.sx', '.ca', '.澳門', '.za', '.th', '.ci', '.政府', '.ru', '.et', '.tf', '.tk', '.gb', '.nr', '.ve', '.ch', '.tv', '.bb', '.工行', '.ss', '.is', '.tg', '.mw', '.vn', '.rs', '.pw', '.lr', '.cc', '.机构', '.ws', '.gt', '.tt', '.al', '.商标', '.pl', '.co', '.mu', '.sk', '.ye', '.bf', '.lk', '.dj', '.tj', '.pt', '.gu', '.cn', '.tl', '.af', '.商店', '.ua', '.网络', '.at', '.am', '.to', '.ag', '.ne', '.mf', '.餐厅', '.hk', '.bi', '.tc', '.fk', '.sc', '

# 3. Column 5 and 6.

In [8]:
original_bright_data_df = copy.deepcopy(bright_data_df)

## 3.1. Functions for TLD+1

In [13]:
def is_float(a_string):
    try:
        float(a_string)
        return True
    except ValueError:
        return False

In [14]:
def true_fld(url):
    if url is not None and url.count(".") > 1 and url[-3:] not in two_letter_tld_set:
        reversed_url = url[::-1]
        last_dot_index = reversed_url.find(".")
        second_to_the_last_dot_index = reversed_url[last_dot_index+1:].find(".")
        second_to_the_last_dot_index += last_dot_index + 1
        return url[-second_to_the_last_dot_index:]
    else:
        return url

## 3.2. Create TLD + 1

In [35]:
initial_domain_tld_plus_one_l = []
target_domain_tld_plus_one_l = []
target_domain_index_ip_d = {}

for i in range(len(bright_data_df)):
    if bright_data_df["asn"][i] == "google":
        initial_domain_tld_plus_one_l.append(None)
        target_domain_tld_plus_one_l.append(None)
    else:
        # Initial domain
        initial_domain_tld_plus_one_s = re.search("https(.*)-DNS_Requests.txt", bright_data_df["initial_domain_filename"][i]).group(1)
        initial_domain_fld_s = get_fld(initial_domain_tld_plus_one_s, fix_protocol=True)
        initial_domain_tld_plus_one_l.append(true_fld(initial_domain_fld_s))

        # Target domain
        target_domain_tld_plus_one_s = re.search("(\w+\.\w+)$", bright_data_df["target_domain"][i]).group(1)
        if is_float(target_domain_tld_plus_one_s):
            target_domain_tld_plus_one_l.append(None)
            target_domain_index_ip_d[i] = bright_data_df["target_domain"][i]
        else:
            target_domain_fld_s = get_fld(bright_data_df["target_domain"][i], fix_protocol=True)
            target_domain_tld_plus_one_l.append(true_fld(target_domain_fld_s))


print(initial_domain_tld_plus_one_l[20:30])
print(target_domain_tld_plus_one_l[20:30])
print(len(target_domain_index_ip_d))
print(target_domain_index_ip_d)

[None, None, None, 'njuskalo.hr', '24sata.hr', '24sata.hr', 'roblox.com', 'gloria.hr', '24sata.hr', 'srce.hr']
[None, None, None, 'gemius.pl', 'amazon-adsystem.com', 'tinypass.com', 'rbxcdn.com', 'ampproject.org', 'yahoo.com', 'unizg.hr']
19
{108771: '192.168.8.1', 114348: '172.31.2.38', 640640: '192.168.8.1', 650237: '192.168.8.1', 660628: '188.191.200.4', 661050: '192.168.8.1', 674412: '192.168.8.1', 679604: '192.168.8.1', 734568: '192.168.8.1', 738151: '192.168.8.1', 749609: '192.168.8.1', 753842: '192.168.8.1', 758287: '192.168.8.1', 761291: '192.168.8.1', 766332: '192.168.8.1', 795966: '192.168.8.1', 797925: '192.168.8.1', 808137: '192.168.8.1', 814141: '192.168.8.1'}


## 3.3. Manual checks

In [36]:
print(bright_data_df["target_domain"][181675])
print(target_domain_tld_plus_one_l[181675])

d1z2jf7jlzjs58.cloudfront.net
cloudfront.net


In [37]:
print(bright_data_df["target_domain"][108771])
print(target_domain_tld_plus_one_l[108771])

192.168.8.1
None


In [38]:
print(bright_data_df["target_domain"][52])
print(target_domain_tld_plus_one_l[52])

s.amazon-adsystem.com
amazon-adsystem.com


In [39]:
initial_domain_multiple_dot_set = set()

for tld_plus_one in initial_domain_tld_plus_one_l:
    if tld_plus_one is not None and tld_plus_one.count(".") > 1:
        initial_domain_multiple_dot_set.add(tld_plus_one)
        
print(len(initial_domain_multiple_dot_set))
print(initial_domain_multiple_dot_set)

9
{'agenciatributaria.gob.es', 'bbc.co.uk', 'dailymail.co.uk', 'amazon.co.uk', 'impots.gouv.fr', 'businessinsider.com.pl', 'vulcan.net.pl', 'jbzd.com.pl', 'zamg.ac.at'}


In [40]:
target_domain_multiple_dot_set = set()

for tld_plus_one in target_domain_tld_plus_one_l:
    if tld_plus_one is not None and tld_plus_one.count(".") > 1:
        target_domain_multiple_dot_set.add(tld_plus_one)
        
print(len(target_domain_multiple_dot_set))
print(target_domain_multiple_dot_set)

25
{'businessinsider.com.pl', 'guardianapps.co.uk', 'agenciatributaria.gob.es', 'bbc.co.uk', 'google.com.sv', 'dailymail.co.uk', 'reachgeneric.co.uk', 'impots.gouv.fr', 'google.co.uk', 'google.com.ua', 'bbci.co.uk', 'zamg.ac.at', 'scottishdailyexpress.co.uk', 'dmgmediaprivacy.co.uk', 'mirror.co.uk', 'amazon.co.uk', 'guim.co.uk', 'google.com.tr', 'adtarget.com.tr', 'adskeeper.co.uk', 'jbzd.com.pl', 'anm.co.uk', 'google.com.ng', 'google.com.ar', 'vulcan.net.pl'}


## 3.4. Add new columns

In [43]:
bright_data_df["initial_domain_tld_plus_one"] = initial_domain_tld_plus_one_l
bright_data_df["target_domain_tld_plus_one"] = target_domain_tld_plus_one_l

In [44]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,amazon-adsystem.com
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr


In [45]:
bright_data_df.shape

(1075319, 6)

In [46]:
bright_data_6col_df = copy.deepcopy(bright_data_df)

# 4. Internet Measurement Tools

## 4.1. dig: (url to) hostname to ip

In [49]:
initial_domain_hostname_set = set(bright_data_df["initial_domain_tld_plus_one"])
target_domain_hostname_set = set(bright_data_df["target_domain_tld_plus_one"])

In [50]:
initial_domain_hostname_set == target_domain_hostname_set

False

In [51]:
start_time = time.time()
initial_domain_hostname_ip_d = {}

for hostname_s in initial_domain_hostname_set:
    initial_domain_hostname_ip_d[hostname_s] = []

    if hostname_s is not None:
        cmd = "dig +short " + hostname_s
        ip_l = os.popen(cmd).read().split("\n")

        for ip in ip_l:
            if ip != "":
                try:
                    ipaddress.ip_address(ip)
                    initial_domain_hostname_ip_d[hostname_s].append(ip)
                except ValueError:
                    print(f"{hostname_s} has an invalid address/netmask at {ip}")
                except:
                    print("Usage : %s  ip" % ip)
                

elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))
print(list(initial_domain_hostname_ip_d.items())[:10])

0:02:48.585577
[('gong.bg', ['87.120.176.15']), ('alza.sk', []), ('lidl.cz', ['104.75.103.190']), ('fandom.com', []), ('kauppalehti.fi', []), ('rtl.be', []), ('ku.dk', ['130.226.237.173']), ('expressen.se', ['3.70.153.12', '52.58.103.55', '35.156.23.11']), ('seiska.fi', []), ('telia.fi', [])]


In [52]:
start_time = time.time()
target_domain_hostname_ip_d = {}

for hostname_s in target_domain_hostname_set:
    target_domain_hostname_ip_d[hostname_s] = []

    if hostname_s is not None:
        cmd = "dig +short " + hostname_s
        ip_l = os.popen(cmd).read().split("\n")

        for ip in ip_l:
            if ip != "":
                try:
                    ipaddress.ip_address(ip)
                    target_domain_hostname_ip_d[hostname_s].append(ip)
                except ValueError:
                    print(f"{hostname_s} has an invalid address/netmask at {ip}")
                except:
                    print("Usage : %s  ip" % ip)
                
                
elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))
print(list(target_domain_hostname_ip_d.items())[:10])

detrimentalunits.com has an invalid address/netmask at ;; connection timed out; no servers could be reached
0:08:01.171528
[('msecnd.net', []), ('cognitivlabs.com', []), ('midas-network.com', ['104.22.48.110', '172.67.41.37', '104.22.49.110']), ('fandom.com', ['151.101.192.194', '151.101.0.194', '151.101.64.194', '151.101.128.194']), ('234doo.com', ['104.21.21.37', '172.67.196.93']), ('telia.fi', []), ('ulisboa.pt', ['194.117.3.27']), ('zencdn.net', []), ('truste.com', []), ('fastclick.net', [])]


## 4.2. whois: ip to AS Number (ASN) and AS Name (AS)

In [54]:
print(len(initial_domain_hostname_set) == len(initial_domain_hostname_ip_d.keys()))
print(len(target_domain_hostname_set) == len(target_domain_hostname_ip_d.keys()))

True
True


In [55]:
start_time = time.time()
initial_domain_hostname_asn_d = {}
initial_domain_hostname_as_d = {}

for hostname, ip_l in initial_domain_hostname_ip_d.items():

    # Sanity check.
    # If ever the length of ASN and AS do not match, then break this loop
    if len(initial_domain_hostname_asn_d.values()) != len(initial_domain_hostname_as_d.values()):
        print(hostname, ip)
        break

    initial_domain_hostname_asn_d[hostname] = []
    initial_domain_hostname_as_d[hostname] = []
    
    for ip in ip_l:
        cmd = f"whois -h whois.cymru.com {ip}"
        # try:
        cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

        for index, item in enumerate(cymru_l):
            if index == 0:
                initial_domain_hostname_asn_d[hostname].append(item.strip())
            if index == 2:
                initial_domain_hostname_as_d[hostname].append(item.strip())
        # except:
        #     cymru_l = os.popen(cmd).read()
        #     print(cymru_l)
        #     break



elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))
print(list(initial_domain_hostname_asn_d.items())[:10])
print(list(initial_domain_hostname_as_d.items())[:10])

0:00:56.785191
[('gong.bg', ['34224']), ('alza.sk', []), ('lidl.cz', ['16625']), ('fandom.com', []), ('kauppalehti.fi', []), ('rtl.be', []), ('ku.dk', ['1835']), ('expressen.se', ['16509', '16509', '16509']), ('seiska.fi', []), ('telia.fi', [])]
[('gong.bg', ['NETERRA-AS, BG']), ('alza.sk', []), ('lidl.cz', ['AKAMAI-AS, US']), ('fandom.com', []), ('kauppalehti.fi', []), ('rtl.be', []), ('ku.dk', ['FSKNET-DK Forskningsnettet - Danish network for Research and Education, EU']), ('expressen.se', ['AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US']), ('seiska.fi', []), ('telia.fi', [])]


In [57]:
start_time = time.time()
target_domain_hostname_asn_d = {}
target_domain_hostname_as_d = {}

for hostname, ip_l in target_domain_hostname_ip_d.items():

    # Sanity check.
    # If ever the length of ASN and AS do not match, then break this loop
    if len(target_domain_hostname_asn_d.values()) != len(target_domain_hostname_as_d.values()):
        print(hostname, ip)
        break

    target_domain_hostname_asn_d[hostname] = []
    target_domain_hostname_as_d[hostname] = []
    
    for ip in ip_l:
        cmd = f"whois -h whois.cymru.com {ip}"
        # try:
        cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

        for index, item in enumerate(cymru_l):
            if index == 0:
                target_domain_hostname_asn_d[hostname].append(item.strip())
            if index == 2:
                target_domain_hostname_as_d[hostname].append(item.strip())
        # except:
        #     cymru_l = os.popen(cmd).read()
        #     print(cymru_l)
        #     break


for idx, ip in target_domain_index_ip_d.items():
    
    # Sanity check.
    # If ever the length of ASN and AS do not match, then break this loop
    if len(target_domain_hostname_asn_d.values()) != len(target_domain_hostname_as_d.values()):
        print(hostname, ip)
        break

    target_domain_hostname_asn_d[idx] = []
    target_domain_hostname_as_d[idx] = []
    
    cmd = f"whois -h whois.cymru.com {ip}"
    # try:
    cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

    for index, item in enumerate(cymru_l):
        if index == 0:
            target_domain_hostname_asn_d[idx].append(item.strip())
        if index == 2:
            target_domain_hostname_as_d[idx].append(item.strip())


elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))
print(list(target_domain_hostname_asn_d.items())[:10])
print(list(target_domain_hostname_as_d.items())[:10])

0:03:29.313432
[('msecnd.net', []), ('cognitivlabs.com', []), ('midas-network.com', ['13335', '13335', '13335']), ('fandom.com', ['54113', '54113', '54113', '54113']), ('234doo.com', ['13335', '13335']), ('telia.fi', []), ('ulisboa.pt', ['1930']), ('zencdn.net', []), ('truste.com', []), ('fastclick.net', [])]
[('msecnd.net', []), ('cognitivlabs.com', []), ('midas-network.com', ['CLOUDFLARENET, US', 'CLOUDFLARENET, US', 'CLOUDFLARENET, US']), ('fandom.com', ['FASTLY, US', 'FASTLY, US', 'FASTLY, US', 'FASTLY, US']), ('234doo.com', ['CLOUDFLARENET, US', 'CLOUDFLARENET, US']), ('telia.fi', []), ('ulisboa.pt', ['RCCN Fundacao para a Ciencia e a Tecnologia, I.P., PT']), ('zencdn.net', []), ('truste.com', []), ('fastclick.net', [])]


In [59]:
print(list(target_domain_index_ip_d.items())[10:])
print(list(target_domain_hostname_asn_d.items())[-10:])
print(list(target_domain_hostname_as_d.items())[-10:])

[(749609, '192.168.8.1'), (753842, '192.168.8.1'), (758287, '192.168.8.1'), (761291, '192.168.8.1'), (766332, '192.168.8.1'), (795966, '192.168.8.1'), (797925, '192.168.8.1'), (808137, '192.168.8.1'), (814141, '192.168.8.1')]
[(738151, ['NA']), (749609, ['NA']), (753842, ['NA']), (758287, ['NA']), (761291, ['NA']), (766332, ['NA']), (795966, ['NA']), (797925, ['NA']), (808137, ['NA']), (814141, ['NA'])]
[(738151, ['NA']), (749609, ['NA']), (753842, ['NA']), (758287, ['NA']), (761291, ['NA']), (766332, ['NA']), (795966, ['NA']), (797925, ['NA']), (808137, ['NA']), (814141, ['NA'])]


## 4.3. AS Number to AS2Org_OrgID

In [61]:
asn_orgid_d = {}
orgid_orgname_d = {}

with open('./20220701.as-org2info.txt', 'r') as caida_file:
    for line in caida_file.readlines():
        if not line.startswith("#"):
            items = line.strip().split("|")
            if len(items) == 5:
                orgid_orgname_d[items[0]] = items[2]
            elif len(items) == 6:
                asn_orgid_d[int(items[0])] = items[3]
            else:
                assert False
                
print(list(asn_orgid_d.items())[:10])
print()
print(list(orgid_orgname_d.items())[:10])

[(1, 'LPL-141-ARIN'), (2, 'UNIVER-19-ARIN'), (3, 'MIT-2-ARIN'), (4, 'USC-32-ARIN'), (5, 'WGL-117-ARIN'), (6, 'ATOS-Z-ARIN'), (7, 'ORG-TDSA4-RIPE'), (8, 'RICEUN-ARIN'), (9, 'CARNEG-Z-ARIN'), (10, 'CCICC-ARIN')]

[('111S-ARIN', 'One Eleven Internet Services'), ('1800CO-2-ARIN', '1-800 Contacts, Inc.'), ('1800FL-ARIN', '1-800-Flowers.com, Inc.'), ('1800H-ARIN', '1-800-HOSTING, Inc.'), ('1881CS-ARIN', '1881CS/XPC'), ('1FBU-ARIN', '1st Financial Bank USA'), ('1GPUC-ARIN', '1798 Global Partners (USA), Corp.'), ('1STP-ARIN', 'FIRST STEP INTERNET, LLC'), ('1UWEB-ARIN', '1U Web, INC.'), ('280INC-ARIN', '280, Inc.')]


In [62]:
initial_domain_hostname_orgid_d = {}
initial_domain_hostname_orgname_d = {}

for hostname, asn_l in initial_domain_hostname_asn_d.items():

    initial_domain_hostname_orgid_d[hostname] = set()
    initial_domain_hostname_orgname_d[hostname] = set()

    for asn in asn_l:        
        try:
            initial_domain_hostname_orgid_d[hostname].add(asn_orgid_d[int(asn)])
            initial_domain_hostname_orgname_d[hostname].add(orgid_orgname_d[asn_orgid_d[int(asn)]])
        except:
            print(f"AS number {asn} does not exist")
    
print(len(initial_domain_hostname_asn_d.values()) == len(initial_domain_hostname_orgid_d.values()))
print(list(initial_domain_hostname_orgid_d.items())[:10])
print(len(initial_domain_hostname_asn_d.values()) == len(initial_domain_hostname_orgname_d.values()))
print(list(initial_domain_hostname_orgname_d.items())[:10])

True
[('gong.bg', {'ORG-NL38-RIPE'}), ('alza.sk', set()), ('lidl.cz', {'AKAMAI-ARIN'}), ('fandom.com', set()), ('kauppalehti.fi', set()), ('rtl.be', set()), ('ku.dk', {'@aut-1835-RIPE'}), ('expressen.se', {'AMAZON-4-ARIN'}), ('seiska.fi', set()), ('telia.fi', set())]
True
[('gong.bg', {'Neterra Ltd.'}), ('alza.sk', set()), ('lidl.cz', {'Akamai Technologies, Inc.'}), ('fandom.com', set()), ('kauppalehti.fi', set()), ('rtl.be', set()), ('ku.dk', {'Forskningsnettet - Danish network for Research and Education'}), ('expressen.se', {'Amazon.com, Inc.'}), ('seiska.fi', set()), ('telia.fi', set())]


In [65]:
target_domain_hostname_orgid_d = {}
target_domain_hostname_orgname_d = {}

for hostname, asn_l in target_domain_hostname_asn_d.items():

    target_domain_hostname_orgid_d[hostname] = set()
    target_domain_hostname_orgname_d[hostname] = set()

    for asn in asn_l:
        if asn != "NA":
            try:
                target_domain_hostname_orgid_d[hostname].add(asn_orgid_d[int(asn)])
                target_domain_hostname_orgname_d[hostname].add(orgid_orgname_d[asn_orgid_d[int(asn)]])
            except:
                print(f"AS number {asn} does not exist")
    
print(len(target_domain_hostname_asn_d.values()) == len(target_domain_hostname_orgid_d.values()))
print(list(target_domain_hostname_orgid_d.items())[:10])
print(len(target_domain_hostname_asn_d.values()) == len(target_domain_hostname_orgname_d.values()))
print(list(target_domain_hostname_orgname_d.items())[:10])

AS number 60922 does not exist
True
[('msecnd.net', set()), ('cognitivlabs.com', set()), ('midas-network.com', {'CLOUD14-ARIN'}), ('fandom.com', {'SKYCA-3-ARIN'}), ('234doo.com', {'CLOUD14-ARIN'}), ('telia.fi', set()), ('ulisboa.pt', {'ORG-FPAC1-RIPE'}), ('zencdn.net', set()), ('truste.com', set()), ('fastclick.net', set())]
True
[('msecnd.net', set()), ('cognitivlabs.com', set()), ('midas-network.com', {'Cloudflare, Inc.'}), ('fandom.com', {'Fastly, Inc.'}), ('234doo.com', {'Cloudflare, Inc.'}), ('telia.fi', set()), ('ulisboa.pt', {'Fundacao para a Ciencia e a Tecnologia, I.P.'}), ('zencdn.net', set()), ('truste.com', set()), ('fastclick.net', set())]


# 5. First Party vs Third Party

## 5.1. Method: TLD+1 matching

In [66]:
print(len(initial_domain_tld_plus_one_l) == len(target_domain_tld_plus_one_l))
print(len(initial_domain_tld_plus_one_l) == bright_data_df.shape[0])

True
True


In [70]:
tld_plus_one_first_party_l = bright_data_df.shape[0] * [None]

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] is not None and target_domain_tld_plus_one_l[i] is not None and initial_domain_tld_plus_one_l[i] == target_domain_tld_plus_one_l[i]:
        tld_plus_one_first_party_l[i] = True
        
print(len(tld_plus_one_first_party_l) == len(initial_domain_tld_plus_one_l))
print(tld_plus_one_first_party_l.count(True))
print(f"At least {round(tld_plus_one_first_party_l.count(True) / len(tld_plus_one_first_party_l) * 100, 5)} % is 1st party.")
print(tld_plus_one_first_party_l[50:60])

True
176550
At least 16.41838 % is 1st party.
[True, None, None, None, None, None, None, None, None, None]


## 5.2. Method: AS (AS Name)

In [71]:
match_hostname_as_d = {}

for initial_hostname, initial_as in initial_domain_hostname_as_d.items():
    if target_domain_hostname_as_d[initial_hostname] == initial_as:
        match_hostname_as_d[initial_hostname] = True
        
print(len(match_hostname_as_d) < len(initial_domain_hostname_as_d))
print(list(match_hostname_as_d.items())[:10])

True
[('gong.bg', True), ('lidl.cz', True), ('rtl.be', True), ('ku.dk', True), ('expressen.se', True), ('seiska.fi', True), ('telia.fi', True), ('jofogas.hu', True), ('irishmirror.ie', True), ('worten.pt', True)]


## 5.3. Method: ASN (AS Number)

In [72]:
match_hostname_asn_d = {}

for initial_hostname, initial_asn in initial_domain_hostname_asn_d.items():
    if target_domain_hostname_asn_d[initial_hostname] == initial_asn:
        match_hostname_asn_d[initial_hostname] = True
        
print(len(match_hostname_asn_d) < len(initial_domain_hostname_asn_d))
print(list(match_hostname_asn_d.items())[:10])

True
[('gong.bg', True), ('lidl.cz', True), ('rtl.be', True), ('ku.dk', True), ('expressen.se', True), ('seiska.fi', True), ('telia.fi', True), ('jofogas.hu', True), ('irishmirror.ie', True), ('worten.pt', True)]


## 5.4. Method: Org ID

In [74]:
match_hostname_orgid_d = {}

for initial_hostname, initial_orgid in initial_domain_hostname_orgid_d.items():
    if target_domain_hostname_orgid_d[initial_hostname] == initial_orgid:
        match_hostname_orgid_d[initial_hostname] = True
        
print(len(match_hostname_orgid_d) < len(initial_domain_hostname_orgid_d))
print(list(match_hostname_orgid_d.items())[:10])

True
[('gong.bg', True), ('lidl.cz', True), ('rtl.be', True), ('ku.dk', True), ('expressen.se', True), ('seiska.fi', True), ('telia.fi', True), ('jofogas.hu', True), ('irishmirror.ie', True), ('worten.pt', True)]


## 5.5. Method: Org Name

In [75]:
match_hostname_orgname_d = {}

for initial_hostname, initial_orgname in initial_domain_hostname_orgname_d.items():
    if target_domain_hostname_orgname_d[initial_hostname] == initial_orgname:
        match_hostname_orgname_d[initial_hostname] = True
        
print(len(match_hostname_orgname_d) < len(initial_domain_hostname_orgname_d))
print(list(match_hostname_orgname_d.items())[:10])

True
[('gong.bg', True), ('lidl.cz', True), ('rtl.be', True), ('ku.dk', True), ('expressen.se', True), ('seiska.fi', True), ('telia.fi', True), ('jofogas.hu', True), ('irishmirror.ie', True), ('worten.pt', True)]


# 6. A combined df: Add columns

## 6.1. tld_plus_one_first_party_l

In [78]:
bright_data_df.shape

(1075319, 6)

In [77]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,amazon-adsystem.com
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr


In [79]:
bright_data_df["tld_plus_one_first_party"] = tld_plus_one_first_party_l

In [80]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,amazon-adsystem.com,None
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,s-onetag.com,None
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,s-onetag.com,None


In [81]:
bright_data_df.shape

(1075319, 7)

In [82]:
bright_data_7col_df = copy.deepcopy(bright_data_df)

## 6.2. as_name_first_party_l

In [83]:
as_name_first_party_l = bright_data_df.shape[0] * [None]
match_as_name_set = set(match_hostname_as_d.keys())
match_as_name_set.remove(None)

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] in match_as_name_set:
        as_name_first_party_l[i] = True
        
print(as_name_first_party_l.count(None) + as_name_first_party_l.count(True) == bright_data_df.shape[0])
print(f"{round(as_name_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% is the first party based on AS.")

True
73.45179% is the first party based on AS.


In [84]:
bright_data_df["as_name_first_party"] = as_name_first_party_l

In [85]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,None
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None,True
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,amazon-adsystem.com,None,True
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None,True
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None,None
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None,True
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None,True
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,s-onetag.com,None,None
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None,True
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,s-onetag.com,None,None


In [86]:
bright_data_df.shape

(1075319, 8)

In [87]:
bright_data_8col_df = copy.deepcopy(bright_data_df)

## 6.3. as_number_first_party_l

In [88]:
as_number_first_party_l = bright_data_df.shape[0] * [None]
match_as_number_set = set(match_hostname_asn_d.keys())
match_as_number_set.remove(None)

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] in match_as_number_set:
        as_number_first_party_l[i] = True
        
print(as_number_first_party_l.count(None) + as_number_first_party_l.count(True) == bright_data_df.shape[0])
print(f"{round(as_number_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% is the first party based on ASN.")

True
73.45179% is the first party based on ASN.


In [89]:
bright_data_df["as_number_first_party"] = as_number_first_party_l

In [90]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,None,None
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None,True,True
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,amazon-adsystem.com,None,True,True
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None,True,True
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None,None,None
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None,True,True
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None,True,True
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,s-onetag.com,None,None,None
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None,True,True
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,s-onetag.com,None,None,None


In [91]:
bright_data_df.shape

(1075319, 9)

## 6.4. org_id_first_party_l

In [92]:
bright_data_9col_df = copy.deepcopy(bright_data_df)

In [93]:
org_id_first_party_l = bright_data_df.shape[0] * [None]
match_orgid_set = set(match_hostname_orgid_d.keys())
match_orgid_set.remove(None)

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] in match_orgid_set:
        org_id_first_party_l[i] = True
        
print(org_id_first_party_l.count(None) + org_id_first_party_l.count(True) == bright_data_df.shape[0])
print(f"{round(org_id_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% is the first party based on Org ID.")

True
73.79884% is the first party based on Org ID.


In [94]:
bright_data_df["org_id_first_party"] = org_id_first_party_l

In [95]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,None,None,None
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None,True,True,True
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,amazon-adsystem.com,None,True,True,True
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None,True,True,True
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None,None,None,None
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None,True,True,True
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None,True,True,True
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,s-onetag.com,None,None,None,None
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None,True,True,True
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,s-onetag.com,None,None,None,None


In [96]:
bright_data_df.shape

(1075319, 10)

In [97]:
bright_data_10col_df = copy.deepcopy(bright_data_df)

## 6.5. org_name_first_party_l

In [98]:
org_name_first_party_l = bright_data_df.shape[0] * [None]
match_orgid_set = set(match_hostname_orgname_d.keys())
match_orgid_set.remove(None)

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] in match_orgid_set:
        org_name_first_party_l[i] = True
        
print(org_name_first_party_l.count(None) + org_name_first_party_l.count(True) == bright_data_df.shape[0])
print(f"{round(org_name_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on Org Name.")

True
73.79884% are the first party based on Org Name.


In [99]:
bright_data_df["org_name_first_party"] = org_name_first_party_l

In [100]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,None,None,None,None
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None,True,True,True,True
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,amazon-adsystem.com,None,True,True,True,True
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None,True,True,True,True
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None,None,None,None,None
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None,True,True,True,True
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None,True,True,True,True
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,s-onetag.com,None,None,None,None,None
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None,True,True,True,True
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,s-onetag.com,None,None,None,None,None


In [101]:
bright_data_df.shape

(1075319, 11)

In [102]:
bright_data_11col_df = copy.deepcopy(bright_data_df)

## 6.6. Altogether

In [103]:
bright_data_df['first_party'] = np.select([bright_data_df.iloc[:, 6:10].any(axis="columns")], [True], default=None)

In [105]:
print(f"{round(list(bright_data_df['first_party']).count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party overall.")

78.47987% are the first party overall.


In [106]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party,first_party
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None,None,None,None,None,None,None
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None,None,None,None,None,None,None
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None,None,None,None,None,None,None
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl,None,True,True,True,True,True
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,amazon-adsystem.com,None,True,True,True,True,True
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com,None,True,True,True,True,True
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com,None,None,None,None,None,None
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org,None,True,True,True,True,True
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com,None,True,True,True,True,True
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr,None,None,None,None,None,None


In [109]:
bright_data_df.shape

(1075319, 12)

In [107]:
bright_data_12col_df = copy.deepcopy(bright_data_df)

In [108]:
os.chdir(output_path_s)
print(os.getcwd())
bright_data_df.to_csv("domains_bright_data_12col.csv")

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/output


# 7. Parser list for trackers

## 7.1. Read TXT: easyprivacy.txt

In [110]:
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/input


In [111]:
easyprivacy_df = pd.read_csv("easyprivacy.txt", on_bad_lines="skip")

In [112]:
easyprivacy_l = easyprivacy_df["[Adblock Plus 1.1]"].values.tolist()

In [113]:
parser_l = [parser for parser in easyprivacy_l if parser[0] != "!"]

## 7.2. Parse: This step takes a long time!

In [114]:
start_time = time.time()
is_target_domain_tracker_l = []

for tld_plus_one in target_domain_tld_plus_one_l:
    is_tracker = False
    if tld_plus_one is not None:        
        for parser in parser_l:
            if tld_plus_one in parser:
                is_tracker = True
    is_target_domain_tracker_l.append(is_tracker)

elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))

0:31:24.105996


In [115]:
print(len(target_domain_tld_plus_one_l) == len(is_target_domain_tracker_l))
print(is_target_domain_tracker_l[20:30])
print(f"{round(np.count_nonzero(np.array(is_target_domain_tracker_l)) / len(is_target_domain_tracker_l) * 100, 5)}% of target domains are trackers.")

True
[False, False, False, True, False, True, False, True, True, False]
66.2887% of target domains are trackers.


## 7.3. Add a column

In [117]:
bright_data_df["is_target_domain_tracker"] = is_target_domain_tracker_l

In [119]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party,first_party,is_target_domain_tracker
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None,None,None,None,None,None,None,False
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None,None,None,None,None,None,None,False
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None,None,None,None,None,None,None,False
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl,None,True,True,True,True,True,True
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,amazon-adsystem.com,None,True,True,True,True,True,False
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com,None,True,True,True,True,True,True
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com,None,None,None,None,None,None,False
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org,None,True,True,True,True,True,True
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com,None,True,True,True,True,True,True
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr,None,None,None,None,None,None,False


In [120]:
bright_data_df.shape

(1075319, 13)

In [121]:
bright_data_13col_df = copy.deepcopy(bright_data_df)

In [122]:
os.chdir(output_path_s)
print(os.getcwd())
bright_data_df.to_csv("domains_bright_data_13col.csv")

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/output


# 8. Functions for Google

In [123]:
def whois(ip_l):
    as_name_l = []
    as_number_l = []
    
    for ip in ip_l:
        cmd = f"whois -h whois.cymru.com {ip}"
        try:
            cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

            for index, item in enumerate(cymru_l):
                if index == 0:
                    as_number_l.append(item.strip())
                if index == 2:
                    as_name_l.append(item.strip())
        except:
            print(f"Run whois again for this ip: {ip}")
                
    return as_name_l, as_number_l 

In [124]:
def get_descriptive_statistics_about_as_name_and_as_number(as_name_l, as_number_l):
    print(f"The length of AS Name is {len(as_name_l)}\n")
        
    print(f"The length of AS Number is {len(as_number_l)}\n")
    
    if len(as_name_l) == len(as_number_l):
        print("AS Name and AS Number have the same length.\n")
    elif len(as_name_l) > len(as_number_l):
        print("AS Name has a longer length than AS Number.\n")
    else:
        print("AS Number has a longer length than AS Name.\n")
   
    if len(as_name_l) == len(set(as_name_l)):
        print("No duplicates in AS Name.\n")
    else:
        print(f"Duplicates in AS Name. set(as_name_l) = {set(as_name_l)}\n")

    if len(as_number_l) == len(set(as_number_l)):
        print("No duplicates in AS Number.\n")
    else:
        print(f"Duplicates in AS Number. set(as_number_l) = {set(as_number_l)}\n")

# 9. Google.com

In [125]:
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/input


## 9.1. dig: hostname to ip

In [128]:
google_ip_l = []

hostname_s = "google.com"

cmd = "dig +short " + hostname_s
ip_ll = os.popen(cmd).read().split("\n")

for index, ip in enumerate(ip_ll):
    if ip != "":
        google_ip_l.append(ip)
    
print(google_ip_l)

['142.250.188.238']


## 9.2. whois: ip to AS Number and AS Name

In [129]:
google_as_name_l, google_as_number_l = whois(google_ip_l)

In [130]:
get_descriptive_statistics_about_as_name_and_as_number(google_as_name_l, google_as_number_l)

The length of AS Name is 1

The length of AS Number is 1

AS Name and AS Number have the same length.

No duplicates in AS Name.

No duplicates in AS Number.



In [131]:
google_as_name_set, google_as_number_set = set(google_as_name_l), set(google_as_number_l)

## 9.3. AS Number to AS2Org_OrgID

In [132]:
google_orgname_l = []

for as_numbrer in google_as_number_l:
    google_orgname_l.append(asn_orgid_d[int(as_numbrer)])

print(google_orgname_l)

['GOGL-ARIN']


In [133]:
google_orgname_set = set(google_orgname_l)

## 9.4. Summary

In [134]:
print(google_ip_l)
print(google_as_name_set)
print(google_as_number_set)
print(google_orgname_set)

['142.250.188.238']
{'GOOGLE, US'}
{'15169'}
{'GOGL-ARIN'}


## 9.5. Convert a set into a string, since a set has only 1 elements

In [135]:
for google_as_name in google_as_name_set:
    google_as_name_s = google_as_name

In [136]:
for google_as_number in google_as_number_set:
    google_as_number_s = google_as_number

In [137]:
for google_orgname in google_orgname_set:
    google_orgname_s = google_orgname

In [138]:
print(google_as_name_s)
print(google_as_number_s)
print(google_orgname_s)

GOOGLE, US
15169
GOGL-ARIN


# 10. Google Match

## 10.1. Include asn == "google". The initial status.

In [139]:
is_target_domain_google_l = len(target_domain_tld_plus_one_l) * [False]
print(len(is_target_domain_google_l) == bright_data_df.shape[0])

True


In [140]:
index_not_to_check_set = set()

for i in range(len(is_target_domain_google_l)):
    if bright_data_df["asn"][i] == "google":
        is_target_domain_google_l[i] = True
        index_not_to_check_set.add(i)

print(is_target_domain_google_l.count(True) == len(index_not_to_check_set))
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain is Google. This is what the initial dataset is given to me.")

True
0.05924% of target domain is Google. This is what the initial dataset is given to me.


## 10.2. Method: AS (AS Name)

In [142]:
for hostname, as_name_l in target_domain_hostname_as_d.items():
    if google_as_name_s in as_name_l:
        for index, tld_plus_one in enumerate(target_domain_tld_plus_one_l):
            if tld_plus_one == hostname and index not in index_not_to_check_set:
                is_target_domain_google_l[index] = True
                index_not_to_check_set.add(index)

print(is_target_domain_google_l.count(True) == len(index_not_to_check_set))
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain are Google.")

True
12.91747% of target domain are Google.


## 10.3. Method: ASN (AS Number)

In [143]:
for hostname, as_number_l in target_domain_hostname_asn_d.items():
    if google_as_name_s in as_number_l:
        for index, tld_plus_one in enumerate(target_domain_tld_plus_one_l):
            if tld_plus_one == hostname and index not in index_not_to_check_set:
                is_target_domain_google_l[index] = True
                index_not_to_check_set.add(index)

print(is_target_domain_google_l.count(True) == len(index_not_to_check_set))
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain are Google.")

True
12.91747% of target domain are Google.


## 10.4.

## 10.5. Method: Org Name

In [144]:
for hostname, orgname_l in target_domain_hostname_orgname_d.items():
    if google_as_name_s in orgname_l:
        for index, tld_plus_one in enumerate(target_domain_tld_plus_one_l):
            if tld_plus_one == hostname and index not in index_not_to_check_set:
                is_target_domain_google_l[index] = True
                index_not_to_check_set.add(index)

print(is_target_domain_google_l.count(True) == len(index_not_to_check_set))
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain are Google.")

True
12.91747% of target domain are Google.


In [145]:
is_target_domain_google_l[20:30]

[True, True, True, False, False, False, False, True, False, False]

## 10.6. Add a column

In [146]:
bright_data_df["is_target_domain_google"] = is_target_domain_google_l

In [148]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party,first_party,is_target_domain_tracker,is_target_domain_google
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None,None,None,None,None,None,None,False,True
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None,None,None,None,None,None,None,False,True
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None,None,None,None,None,None,None,False,True
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl,None,True,True,True,True,True,True,False
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,amazon-adsystem.com,None,True,True,True,True,True,False,False
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com,None,True,True,True,True,True,True,False
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com,None,None,None,None,None,None,False,False
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org,None,True,True,True,True,True,True,True
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com,None,True,True,True,True,True,True,False
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr,None,None,None,None,None,None,False,False


In [149]:
bright_data_df.shape

(1075319, 14)

In [150]:
bright_data_14col_df = copy.deepcopy(bright_data_df)

In [151]:
os.chdir(output_path_s)
print(os.getcwd())
bright_data_df.to_csv("domains_bright_data_14col.csv")

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/output


# 11. Important Statistics

In [152]:
print(f"{round(as_name_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on AS.")
print(f"{round(as_number_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on ASN.")
print(f"{round(org_id_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on Org ID.")
print(f"{round(org_name_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on Org Name.")

73.45179% are the first party based on AS.
73.45179% are the first party based on ASN.
73.79884% are the first party based on Org ID.
73.79884% are the first party based on Org Name.


In [153]:
print(f"{round(np.count_nonzero(np.array(is_target_domain_tracker_l)) / len(is_target_domain_tracker_l) * 100, 5)}% of target domain are trackers.")
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain are Google.")

66.2887% of target domain are trackers.
12.91747% of target domain are Google.
